# Outlook

In this notebook, using BBRL, we code the DDPG algorithm.

To understand this code, you need to know more about 
[the BBRL interaction model](https://colab.research.google.com/drive/1_yp-JKkxh_P8Yhctulqm0IrLbE41oK1p?usp=sharing).
Then you should run [a first example](https://colab.research.google.com/drive/1Ui481r47fNHCQsQfKwdoNEVrEiqAEokh?usp=sharing)
to see how agents interact.

You also need to understand [details about the
AutoResetGymAgent](https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5?usp=sharing).

The DDPG algorithm is explained in [this video](https://www.youtube.com/watch?v=0D6a0a1HTtc)
and you can also read [the # corresponding slides](http://pages.isir.upmc.fr/~sigaud/teach/ddpg.pdf).

## Installation and Imports

### Installation

The BBRL library is [here](https://github.com/osigaud/bbrl).

We use OmegaConf to that makes it possible that by just defining the `def
run_dqn(cfg):` function and then executing a long `params = {...}` variable at
the bottom of this colab, the code is run with the parameters without calling
an explicit main.

More precisely, the code is run by calling

`config=OmegaConf.create(params)`

`run_dqn(config)`

at the very bottom of the colab, after starting tensorboard.

Below, we import standard python packages, pytorch packages and gymnasium
environments.

In [1]:
# Installs the necessary Python and system libraries
try:
    from easypip import easyimport, easyinstall, is_notebook
except ModuleNotFoundError as e:
    get_ipython().run_line_magic("pip", "install easypip")
    from easypip import easyimport, easyinstall, is_notebook

easyinstall("bbrl>=0.2.2")
easyinstall("swig")
easyinstall("bbrl_gymnasium>=0.2.0")
easyinstall("bbrl_gymnasium[box2d]")
easyinstall("bbrl_gymnasium[classic_control]")
easyinstall("tensorboard")
easyinstall("moviepy")
easyinstall("box2d-kengz")


[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import sys
from pathlib import Path
import math

from moviepy.editor import ipython_display as video_display
import time
from tqdm.auto import tqdm
from typing import Tuple, Optional
from functools import partial

from omegaconf import OmegaConf
import torch
import bbrl_gymnasium

import copy
from abc import abstractmethod, ABC
import torch.nn as nn
import torch.nn.functional as F
from time import strftime
OmegaConf.register_new_resolver(
    "current_time", lambda: strftime("%Y%m%d-%H%M%S"), replace=True
)

In [3]:
# Imports all the necessary classes and functions from BBRL
from bbrl.agents.agent import Agent
from bbrl import get_arguments, get_class, instantiate_class
# The workspace is the main class in BBRL, this is where all data is collected and stored
from bbrl.workspace import Workspace

# Agents(agent1,agent2,agent3,...) executes the different agents the one after the other
# TemporalAgent(agent) executes an agent over multiple timesteps in the workspace, 
# or until a given condition is reached
from bbrl.agents import Agents, TemporalAgent

# ParallelGymAgent is an agent able to execute a batch of gymnasium environments
# with auto-resetting. These agents produce multiple variables in the workspace:
# ’env/env_obs’, ’env/reward’, ’env/timestep’, ’env/terminated’,
# 'env/truncated', 'env/done', ’env/cumulated_reward’, ... 
# 
# When called at timestep t=0, the environments are automatically reset. At
# timestep t>0, these agents will read the ’action’ variable in the workspace at
# time t − 1
from bbrl.agents.gymnasium import GymAgent, ParallelGymAgent, make_env, record_video

# Replay buffers are useful to store past transitions when training
from bbrl.utils.replay_buffer import ReplayBuffer

In [4]:
# Utility function for launching tensorboard
# For Colab - otherwise, it is easier and better to launch tensorboard from
# the terminal
def setup_tensorboard(path):
    path = Path(path)
    answer = ""
    if is_notebook():
        if get_ipython().__class__.__module__ == "google.colab._shell":
            answer = "y"
        while answer not in ["y", "n"]:
                answer = input(f"Do you want to launch tensorboard in this notebook [y/n] ").lower()

    if answer == "y":
        get_ipython().run_line_magic("load_ext", "tensorboard")
        get_ipython().run_line_magic("tensorboard", f"--logdir {path.absolute()}")
    else:
        import sys
        import os
        import os.path as osp
        print(f"Launch tensorboard from the shell:\n{osp.dirname(sys.executable)}/tensorboard --logdir={path.absolute()}")

The [DDPG](https://arxiv.org/pdf/1509.02971.pdf) algorithm is an actor critic
algorithm. We use a simple neural network builder function. This neural
networks plays the role of the actor and the critic.

## Definition of agents

The function below builds a multi-layer perceptron where the size of each layer is given in the `size` list.
We also specify the activation function of neurons at each layer and optionally a different activation function for the final layer.

In [5]:
import torch.nn as nn
def build_mlp(sizes, activation, output_activation=nn.Identity()):
    """Helper function to build a multi-layer perceptron (function from $\mathbb R^n$ to $\mathbb R^p$)
    
    Args:
        sizes (List[int]): the number of neurons at each layer
        activation (nn.Module): a PyTorch activation function (after each layer but the last)
        output_activation (nn.Module): a PyTorch activation function (last layer)
    """
    layers = []
    for j in range(len(sizes) - 1):
        act = activation if j < len(sizes) - 2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j + 1]), act]
    return nn.Sequential(*layers)

The critic is a neural network taking the state $s$ and action $a$ as input,
and its output layer has a unique neuron whose value is the value of being in
that state and performing that action $Q(s,a)$.

As usual, the ```forward(...)``` function is used to write Q-values in the
workspace from time indexes, whereas the ```predict_value(...)``` function` is
used in other contexts, such as plotting a view of the Q function.

In [6]:
class ContinuousQAgent(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim, name="critic"):
        super().__init__()
        self.is_q_function = True
        self.model = build_mlp(
            [state_dim + action_dim] + list(hidden_layers) + [1], activation=nn.ReLU()
        )
        self.with_name(name)

    def with_name(self, name):
        self.name = f"{name}/q_value"
        return self

    def forward(self, t, detach_actions=False):
        obs = self.get(("env/env_obs", t))
        action = self.get(("action", t))
        if detach_actions:
            action = action.detach()
        osb_act = torch.cat((obs, action), dim=1)
        q_value = self.model(osb_act)
        self.set((self.name, t), q_value)

    def predict_value(self, obs, action):
        osb_act = torch.cat((obs, action), dim=0)
        q_value = self.model(osb_act)
        return q_value

The actor is also a neural network, it takes a state $s$ as input and outputs
an action $a$.

In [7]:
class ContinuousDeterministicActor(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim):
        super().__init__()
        layers = [state_dim] + list(hidden_layers) + [action_dim]
        self.model = build_mlp(
            layers, activation=nn.ReLU(), output_activation=nn.Tanh()
        )

    def forward(self, t):
        obs = self.get(("env/env_obs", t))
        action = self.model(obs)
        self.set(("action", t), action)

    def predict_action(self, obs, stochastic):
        assert (
            not stochastic
        ), "ContinuousDeterministicActor cannot provide stochastic predictions"
        return self.model(obs)

### Creating an Exploration method

In the continuous action domain, basic exploration differs from the methods
used in the discrete action domain. Here we generally add some Gaussian noise
to the output of the actor.

In [8]:
from torch.distributions import Normal

In [9]:
class AddGaussianNoise(Agent):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, t, **kwargs):
        act = self.get(("action", t))
        dist = Normal(act, self.sigma)
        action = dist.sample()
        self.set(("action", t), action)

In [the original DDPG paper](https://arxiv.org/pdf/1509.02971.pdf), the
authors rather used the more sophisticated Ornstein-Uhlenbeck noise where
noise is correlated between one step and the next.

In [10]:
class AddOUNoise(Agent):
    """
    Ornstein Uhlenbeck process noise for actions as suggested by DDPG paper
    """

    def __init__(self, std_dev, theta=0.15, dt=1e-2):
        self.theta = theta
        self.std_dev = std_dev
        self.dt = dt
        self.x_prev = 0

    def forward(self, t, **kwargs):
        act = self.get(("action", t))
        x = (
            self.x_prev
            + self.theta * (act - self.x_prev) * self.dt
            + self.std_dev * math.sqrt(self.dt) * torch.randn(act.shape)
        )
        self.x_prev = x
        self.set(("action", t), x)

### Training and evaluation environments

We build two environments: one for training and another one for evaluation.

For training, it is more efficient to use an autoreset agent, as we do not
want to waste time if the task is done in an environment sooner than in the
others.

By contrast, for evaluation, we just need to perform a fixed number of
episodes (for statistics), thus it is more convenient to use a
noautoreset agent with a set of environments and just run one episode in
each environment. Thus we can use the `env/done` stop variable and take the
average over the cumulated reward of all environments.

See [this
notebook](https://colab.research.google.com/drive/1Ui481r47fNHCQsQfKwdoNEVrEiqAEokh?usp=sharing)
for explanations about agents and environment agents.

In [11]:
from typing import Tuple
from bbrl.agents.gymnasium import make_env, GymAgent, ParallelGymAgent
from functools import partial

def get_env_agents(cfg, *, autoreset=True, include_last_state=True) -> Tuple[GymAgent, GymAgent]:
    # Returns a pair of environments (train / evaluation) based on a configuration `cfg`
    
    # Train environment
    train_env_agent = ParallelGymAgent(
        partial(make_env, cfg.gym_env.env_name, autoreset=autoreset),
        cfg.algorithm.n_envs, 
        include_last_state=include_last_state
    ).seed(cfg.algorithm.seed)

    # Test environment
    eval_env_agent = ParallelGymAgent(
        partial(make_env, cfg.gym_env.env_name), 
        cfg.algorithm.nb_evals,
        include_last_state=include_last_state
    ).seed(cfg.algorithm.seed)

    return train_env_agent, eval_env_agent

### Create the DDPG agent

In this function we create the critic and the actor, but also an exploration
agent to add noise and a target critic. The version below does not use a
target actor as it proved hard to tune, but such a target actor is used in the
original paper.

In [12]:
# Create the DDPG Agent
def create_ddpg_agent(cfg, train_env_agent, eval_env_agent):
    obs_size, act_size = train_env_agent.get_obs_and_actions_sizes()
    critic = ContinuousQAgent(
        obs_size, cfg.algorithm.architecture.critic_hidden_size, act_size
    )
    target_critic = copy.deepcopy(critic).with_name("target-critic")
    actor = ContinuousDeterministicActor(
        obs_size, cfg.algorithm.architecture.actor_hidden_size, act_size
    )
    # target_actor = copy.deepcopy(actor) # not used in practice, though described in the paper
    noise_agent = AddGaussianNoise(cfg.algorithm.action_noise) # alternative : AddOUNoise
    tr_agent = Agents(train_env_agent, actor, noise_agent)  
    ev_agent = Agents(eval_env_agent, actor)

    # Get an agent that is executed on a complete workspace
    train_agent = TemporalAgent(tr_agent)
    eval_agent = TemporalAgent(ev_agent)
    return train_agent, eval_agent, actor, critic, target_critic

### The Logger class

The logger is in charge of collecting statistics during the training
process.

Having logging provided under the hood is one of the features allowing you
to save time when using RL libraries like BBRL.

In these notebooks, the logger is defined as `bbrl.utils.logger.TFLogger` so as
to use a tensorboard visualisation (see the parameters part `params = { "logger":{ ...` below).

Note that the BBRL Logger is also saving the log in a readable format such
that you can use `Logger.read_directories(...)` to read multiple logs, create
a dataframe, and analyze many experiments afterward in a notebook for
instance. The code for the different kinds of loggers is available in the
[bbrl/utils/logger.py](https://github.com/osigaud/bbrl/blob/master/src/bbrl/utils/logger.py)
file.

`instantiate_class` is an inner BBRL mechanism. The
`instantiate_class`function is available in the
[`bbrl/__init__.py`](https://github.com/osigaud/bbrl/blob/master/src/bbrl/__init__.py)
file.

In [13]:
from bbrl import instantiate_class

class Logger():

    def __init__(self, cfg):
        self.logger = instantiate_class(cfg.logger)

    def add_log(self, log_string, loss, steps):
        self.logger.add_scalar(log_string, loss.item(), steps)

    # A specific function for RL algorithms having a critic, an actor and an entropy losses
    def log_losses(self, critic_loss, entropy_loss, actor_loss, steps):
        self.add_log("critic_loss", critic_loss, steps)
        self.add_log("entropy_loss", entropy_loss, steps)
        self.add_log("actor_loss", actor_loss, steps)

    def log_reward_losses(self, rewards, nb_steps):
        self.add_log("reward/mean", rewards.mean(), nb_steps)
        self.add_log("reward/max", rewards.max(), nb_steps)
        self.add_log("reward/min", rewards.min(), nb_steps)
        self.add_log("reward/median", rewards.median(), nb_steps)

### Setup the optimizers

We use two separate optimizers to tune the parameters of the actor and the critic separately. That makes it possible to use a different learning rate for the actor and the critic.

In [14]:
# Configure the optimizers
def setup_optimizers(cfg, actor, critic):
    actor_optimizer_args = get_arguments(cfg.actor_optimizer)
    parameters = actor.parameters()
    actor_optimizer = get_class(cfg.actor_optimizer)(parameters, **actor_optimizer_args)
    critic_optimizer_args = get_arguments(cfg.critic_optimizer)
    parameters = critic.parameters()
    critic_optimizer = get_class(cfg.critic_optimizer)(
        parameters, **critic_optimizer_args
    )
    return actor_optimizer, critic_optimizer

### Compute critic loss

Detailed explanations of the function to compute the critic loss when using a NoAutoResetGymAgent are given in [this notebook](https://colab.research.google.com/drive/1raeuB6uUVUpl-4PLArtiAoGnXj0sGjSV?usp=sharing).

The case where we use the AutoResetGymAgent is very similar, but we need to specify that we use the first part of the Q-values (`q_values[0]`) for representing $Q(s_t,a_t)$ and the second part (`q_values[1]`) for representing $Q(s_{t+1},a)$, as these values are stored into a transition model. Then the values used to compute the target in the critic update are stored into ```target_q_values```.

In [15]:
def compute_critic_loss(cfg, reward, must_bootstrap, q_values, target_q_values):
    # Compute temporal difference
    q_next = target_q_values  # a .detach() is not necessary because target_q_values are computed within a with torch.no_grad
    target = (
        reward[1:].squeeze()
        + cfg.algorithm.discount_factor * q_next.squeeze(-1) * must_bootstrap.int()
    )
    # Compute critic loss
    mse = nn.MSELoss()
    critic_loss = mse(target, q_values.squeeze(-1))
    return critic_loss

### Soft parameter updates

To update the target critic, one uses the following equation:
$\theta' \leftarrow \tau \theta + (1- \tau) \theta'$
where $\theta$ is the vector of parameters of the critic, and $\theta'$ is the vector of parameters of the target critic.
The `soft_update_params(...)` function is in charge of performing this soft update.

In [16]:
def soft_update_params(net, target_net, tau):
    for param, target_param in zip(net.parameters(), target_net.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

### Compute actor loss
The actor loss is straightforward. We want the actor to maximize Q-values, thus we minimize the mean of negated Q-values.

In [17]:
def compute_actor_loss(q_values):
    return -q_values.mean()

## Main training loop

### Agent execution

This is the tricky part with BBRL, the one we need to understand in detail.
The difficulty lies in the copy of the last step and the way to deal with the
n_steps return.

The call to `train_agent(workspace, t=1, n_steps=cfg.algorithm.n_timesteps -
1, stochastic=True)` makes the agent run a number of steps in the workspace.
In practice, it calls the
[`__call__(...)`](https://github.com/osigaud/bbrl/blob/master/src/bbrl/agents/agent.py#L59)
function which makes a forward pass of the agent network using the workspace
data and updates the workspace accordingly.

Now, if we start at the first epoch (`epoch=0`), we start from the first step
(`t=0`). But when subsequently we perform the next epochs (`epoch>0`), we must
not forget to cover the transition at the border between the previous epoch
and the current epoch. To avoid this risk, we copy the information from the
last time step of the previous epoch into the first time step of the next
epoch. This is explained in more details in [a previous
notebook](https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5).

A [previous
notebook](https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5)
explains a lot of these details. In particular, read it to understand the
`execute_agents(...)` function, the `transition_workspace =
train_workspace.get_transitions()` line and the computation of
`must_bootstrap`.

Note that we `optimizer.zero_grad()`, `loss.backward()` and `optimizer.step()`
lines. 

`optimizer.zero_grad()` is necessary to cancel all the gradients computed at
the previous iterations

Note the way we count the steps, to properly ignore the steps corresponding to
a transition from an episode to the next. Note also that every
```cfg.algorithm.eval_interval```, we evaluate the current agent and save it
and plot it if its performance is the new best performance.

In [24]:
from bbrl.visu.plot_policies import plot_policy
from bbrl.visu.plot_critics import plot_critic

def run_ddpg(cfg):
    # 1)  Build the  logger
    logger = Logger(cfg)
    best_reward = float('-inf')

    # 2) Create the environment agent
    train_env_agent, eval_env_agent = get_env_agents(cfg)
    
    # 3) Create the DDPG Agent
    (
        train_agent,
        eval_agent,
        actor,
        critic,
        target_critic,
    ) = create_ddpg_agent(cfg, train_env_agent, eval_env_agent)
    ag_actor = TemporalAgent(actor)
    # ag_target_actor = TemporalAgent(target_actor)
    q_agent = TemporalAgent(critic)
    target_q_agent = TemporalAgent(target_critic)

    train_workspace = Workspace()

    # Configure the optimizer
    actor_optimizer, critic_optimizer = setup_optimizers(cfg, actor, critic)
    nb_steps = 0
    tmp_steps = 0

    # Training loop
    pbar = tqdm(range(cfg.algorithm.max_epochs))
    p=1
    for epoch in pbar:
        # Execute the agent in the workspace
        if epoch > 0:
            train_workspace.zero_grad()
            train_workspace.copy_n_last_steps(1)
            train_agent(train_workspace, t=1, n_steps=cfg.algorithm.n_steps)
        else:
            train_agent(train_workspace, t=0, n_steps=cfg.algorithm.n_steps)

        transition_workspace = train_workspace.get_transitions()
        action = transition_workspace["action"]
        nb_steps += action[0].shape[0]

        terminated, reward, action = transition_workspace[
            "env/terminated", "env/reward", "action"
        ]
        # print(f"done {done}, reward {reward}, action {action}")
        
        # Determines whether values of the critic should be propagated
        # True if the episode reached a time limit or if the task was not done
        # See https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5?usp=sharing
        must_bootstrap = ~terminated[1]

        # Critic update
        # compute q_values: at t, we have Q(s,a) from the (s,a) in the RB
        q_agent(transition_workspace, t=0, n_steps=1)
        # print(f"q_values ante : {q_values}")

        with torch.no_grad():
            # replace the action at t+1 in the RB with \pi(s_{t+1}), to compute Q(s_{t+1}, \pi(s_{t+1}) below
            ag_actor(transition_workspace, t=1, n_steps=1)
            # compute q_values: at t+1 we have Q(s_{t+1}, \pi(s_{t+1})
            target_q_agent(transition_workspace, t=1, n_steps=1)
            # q_agent(rb_workspace, t=1, n_steps=1)
        # finally q_values contains the above collection at t=0 and t=1

        q_values, post_q_values = transition_workspace["critic/q_value", "target-critic/q_value"]

        # Compute critic loss
        critic_loss = compute_critic_loss(
            cfg, reward, must_bootstrap, q_values[0], post_q_values[1]
        )
        logger.add_log("critic_loss", critic_loss, nb_steps)
        critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(
            critic.parameters(), cfg.algorithm.max_grad_norm
        )
        critic_optimizer.step()

        # Actor update
        choice = torch.rand(1)

        # Now we determine the actions the current policy would take in the states from the RB
        ag_actor(transition_workspace, t=0, n_steps=1)
        # We determine the Q values resulting from actions of the current policy
        q_agent(transition_workspace, t=0, n_steps=1)
        # and we back-propagate the corresponding loss to maximize the Q values
        q_values = transition_workspace["critic/q_value"]
        actor_loss = compute_actor_loss(q_values)
        logger.add_log("actor_loss", actor_loss, nb_steps)
        # if -25 < actor_loss < 0 and nb_steps > 2e5:
        if choice > p:
            # if -25 < actor_loss < 0 and nb_steps > 2e5:
            actor_optimizer.zero_grad()
            actor_loss.backward()
            torch.nn.utils.clip_grad_norm_(
                actor.parameters(), cfg.algorithm.max_grad_norm
            )
            actor_optimizer.step()
        # decay not given in the article (we go to minimum of 0.01)
        p*=0.999
        p = max(p, 0.01)

        if nb_steps - tmp_steps > cfg.algorithm.eval_interval:
            tmp_steps = nb_steps
            eval_workspace = Workspace()  # Used for evaluation
            eval_agent(eval_workspace, t=0, stop_variable="env/done")
            rewards = eval_workspace["env/cumulated_reward"][-1]
            mean = rewards.mean()
            logger.add_log("reward", mean, nb_steps)
            pbar.set_description(f"nb steps: {nb_steps}, reward: {mean:.3f}, best reward: {best_reward:.3f}")

            if mean > best_reward:
                best_reward = mean

                if cfg.save_best:
                    directory = "./ddpg_agent/"
                    if not os.path.exists(directory):
                        os.makedirs(directory)
                    filename = directory + "ddpg_" + str(mean.item()) + ".agt"
                    eval_agent.save_model(filename)

                if False and cfg.plot_agents:
                    plot_policy(
                        actor,
                        eval_env_agent,
                        best_reward,
                        "",
                        cfg.gym_env.env_name,
                    )
                    plot_critic(
                        q_agent.agent,
                        eval_env_agent,
                        best_reward,
                        "tmp",
                        cfg.gym_env.env_name,
                        input_action=1, # input_action=None will not work for now
                    )

# Definition of the parameters

The logger is defined as `bbrl.utils.logger.TFLogger` so as to use a
tensorboard visualisation.

In [29]:
params={
  "save_best": False,
  # Set to true to have an insight on the learned policy
  # (but slows down the evaluation a lot!)
  "plot_agents": True,
  "logger":{
    "classname": "bbrl.utils.logger.TFLogger",
    "log_dir": "./tblogs/ddpg/ddpg-" + str(time.time()),
    "cache_size": 10000,
    "every_n_seconds": 10,
    "verbose": False,    
    },

  "algorithm":{
    "seed": 1,
    "max_grad_norm": 0.5,
    "epsilon": 0.02,
    "n_envs": 1,
    "n_steps": 100,
    "eval_interval": 2000,
    "nb_evals": 10,
    "max_epochs": 21000,
    "discount_factor": 0.98,
    "buffer_size": 2e5,
    "batch_size": 64,
    "learning_starts": 10000,
    "action_noise": 0.3,
    "architecture":{
        "actor_hidden_size": [12, 12],
        "critic_hidden_size": [12, 12],
        },
  },
  "gym_env":{
    "env_name": "CartPoleContinuous-v1",
  },
  "actor_optimizer":{
    "classname": "torch.optim.SGD",
    "lr": 1e-3,
  },
  "critic_optimizer":{
    "classname": "torch.optim.SGD",
    "lr": 1e-3,
  }
}

## Launching tensorboard to visualize the results

In [32]:
setup_tensorboard("./tblogs")

Do you want to launch tensorboard in this notebook [y/n] y
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [33]:
config=OmegaConf.create(params)
torch.manual_seed(config.algorithm.seed)
run_ddpg(config)

  0%|          | 0/21000 [00:00<?, ?it/s]

## What's next?

Starting from the above version , you should code [the TD3
algorithm](http://proceedings.mlr.press/v80/fujimoto18a/fujimoto18a.pdf).

For that, you need to use two critics (and two target critics) and always take
the minimum output between the two when you ask for the Q-value of a (state,
action) pair.

In more detail, you have to do the following:
- replace the single critic and corresponding target critic with two critics
  and target critics (name them ```critic_1, critic_2, target_critic_1,
  target_critic_2```)
- get the q-values and target q-values corresponding to all these critics.
- then the target q-values you should consider to update the critic should be
  the min over the target q-values at each step (use ```torch.min(...)``` to
  get this min over a sequence of data).
- to update the actor, do it with the q-values of an arbitrarily chosen
  critic, e.g. critic_1.

In [ ]:
# TD3 algorithm

assert False, 'Not implemented yet'


## Experimental comparison

Take an environment where the over-estimation bias may matter, and compare the performance of DDPG and TD3. Visualize the Q-value long before convergence to see whether indeed DDPG overestimates the Q-values with respect to TD3.